- Please start the test in Kaggle https://www.kaggle.com/t/94b12368cbce4c65a2b4ea4be059e312
- demo website: http://realestateevaluator-dev.ap-southeast-1.elasticbeanstalk.com/evaluator/

In [8]:
import pandas as pd
import os
import urllib.request
from sklearn.model_selection import train_test_split

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
import matplotlib.pyplot as plt
from sklearn import preprocessing

import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))

# Any results you write to the current directory are saved as output.


['answer.csv', 'test_submission.csv', '.DS_Store', 'test.csv', 'submission.csv', 'realestateevaluation', 'train.csv']


In [104]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
# df_test.head(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h1> 主要建材

In [65]:
building_material=df_train['主要建材'].astype('str').apply(lambda x:x.lower())
building_material_dict=dict([(name,id)for id ,name in enumerate(sorted(set(df_train['主要建材'].astype('str'))))])
building_material_cat=df_train['主要建材'].astype('str').apply(building_material_dict.get).values
df_train['building_material_cat'] = building_material_cat

<h1> 主要用途

In [101]:
building_purpose_dict=dict([(purpose , id) for id, purpose in enumerate(set(df_train['主要用途'].astype(str)))])
building_purpose_cat=df_train['主要用途'].astype(str).apply(building_purpose_dict.get).values
df_train['building_purpose_cat']=building_purpose_cat
building_purpose_cat

array([1, 1, 5, ..., 5, 5, 5])

<h1>車位類別

In [102]:
car_dict = dict([(typ,idx) for idx,typ in enumerate(set(df_train['車位類別'].astype(str)))])

<h1>鄉鎮市區

In [85]:
districtIndex = list(enumerate(set(df_train["鄉鎮市區"])))
districtDic = {}
for idx, district in districtIndex:
    districtDic[district] = idx
    
    
district_cat = df_train["鄉鎮市區"].apply(districtDic.get)

<h1> 建物型態

In [91]:
buildingIndex = list(enumerate(set(df_train["建物型態"])))
buildingDic = {}
for idx, building in buildingIndex:
    buildingDic[building] = idx
    
    
building_cat = df_train["建物型態"].apply(buildingDic.get)

<h1>建物現況格局-廳 

In [93]:
set(df_train["建物現況格局-廳"])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20, 24, 56, 80}

In [98]:
np.where(df_train["建物現況格局-廳"]==24)

(array([15683]),)

In [99]:
df_train.iloc[15683,:]

index                                                                         15683
主要建材                                                                         鋼筋混凝土造
主要用途                                                                            住家用
交易年月日                                                           0104-10-08 00:00:00
交易標的                                                                      房地(土地+建物)
交易筆棟數                                                                    土地2建物12車位0
備註                                                                              NaN
土地區段位置/建物區段門牌                                                臺北市大安區和平東路二段18巷3弄1~30號
土地移轉總面積(平方公尺)                                                                   678
建物型態                                                                  華廈(10層含以下有電梯)
建物現況格局-廳                                                                         24
建物現況格局-房                                                                    

In [30]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [34]:
districtOneHot[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [59]:
districtOneHot = enc.fit_transform(df_train[["鄉鎮市區"]].values).toarray().reshape(-1,1)

df_train['districtOneHot'] = districtOneHot

ValueError: Length of values does not match length of index

In [51]:
districtOneHot = enc.fit_transform(df_train[["鄉鎮市區"]].values).toarray().reshape(-1,1)

In [47]:
df_train['districtOneHot']

0          (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
1          (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
2          (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
3          (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
4          (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
                               ...                        
69165      (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
69166      (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
69167      (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
69168      (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
69169      (0, 6)\t1.0\n  (1, 1)\t1.0\n  (2, 5)\t1.0\n ...
Name: districtOneHot, Length: 69170, dtype: object

In [2]:
# if 'data' not in os.listdir():
#     os.mkdir('data')
    
# if 'df_realestate.csv' not in os.listdir('data'):
#     url = 'https://s3.amazonaws.com/datasets-jeremy/df_realestate.csv'  
#     urllib.request.urlretrieve(url, os.path.join('data', 'df_realestate.csv')) 
    
# if 'df_realestate_processed.csv' not in os.listdir('data'):
#     url = 'https://s3.amazonaws.com/datasets-jeremy/df_realestate_processed.csv'  
#     urllib.request.urlretrieve(url, os.path.join('data', 'df_realestate_processed.csv'))  

In [87]:
# # before preprocessing
# file = os.path.join('data', 'df_realestate.csv')
# df_realestate = pd.read_csv(file, encoding='big5')
# df_realestate

# # processed
# path = "data//df_realestate_processed.csv"
# df_realestate_processed = pd.read_csv(path)
# X = df_realestate_processed.drop(["price_per_meter", "total_price"], axis=1)
# Y = df_realestate_processed['total_price']


In [88]:
# df_realestate['price_per_ping'] = df_realestate['單價(元/平方公尺)']
# showing_cols = [
#     '主要建材',
#     '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '土地區段位置/建物區段門牌',
#     '土地移轉總面積(平方公尺)', '建物型態', '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛',
#     '建物現況格局-隔間', '建物移轉總面積(平方公尺)', '建築完成年月', '有無管理組織', '移轉層次', '編號',
#     '總樓層數', '車位移轉總面積(平方公尺)', '車位類別', '都市土地使用分區', '鄉鎮市區',
#     '非都市土地使用分區', '非都市土地使用編定', 
#     'num_of_bus_stations_in_100m', 'income_avg', 'income_var', 
#     'location_type', 'low_use_electricity',
#     'nearest_tarin_station', 'nearest_tarin_station_distance',
#     'lat', 'lng', 'price_per_ping'
# ]

# df_realestate = df_realestate[showing_cols]

In [89]:
# print(len(df_realestate))
# df_realestate = df_realestate[pd.notnull(df_realestate['price_per_ping'])]
# print(len(df_realestate))


75203
70670


In [90]:
# train, test = train_test_split(df_realestate, random_state=4242, test_size=1500)
# train = train.reset_index()
# train.loc[:, 'index'] = train.index
# test = test.reset_index()
# test.loc[:, 'index'] = test.index
# answer = test[['index', 'price_per_ping']]
# submission = test[['index', 'price_per_ping']]
# submission.loc[:, 'price_per_ping'] = 0.0
# test = test.drop('price_per_ping', axis=1)

In [94]:
# train.to_csv(os.path.join('data', 'train.csv'), index=False)
# test.to_csv(os.path.join('data', 'test.csv'), index=False)
# answer.to_csv(os.path.join('data', 'answer.csv'), index=False)
# submission.to_csv(os.path.join('data', 'submission.csv'), index=False)


In [96]:
# test_submission = answer
# test_submission['price_per_ping'] = answer['price_per_ping'].mean()

In [99]:
# test_submission.to_csv(os.path.join('data', 'test_submission.csv'), index=False)
